In [1]:
import pandas as pd

In [2]:
icao = pd.read_csv('ICAO_airports.csv')
top = pd.read_csv('top_airports.csv')

1) Use the data supplied of airport codes
(ICAO_airports.csv) and top airports (op_airports.csv)
to make a table of 50 most travelled airports in the US
and containing relevant information, like name, wikipedia
link, latitude, longitude

In [ ]:
import sqlite3

!rm airports.db

connection = sqlite3.connect("icao.db")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE icao (id INTEGER, name TEXT, wikipedia_link TEXT, iata TEXT, latitude_deg FLOAT, longitude_deg FLOAT)"""

cursor.execute(sql_cmd)

In [3]:
icao

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
2,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
3,6526,00AR,heliport,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,00AR,NaN,00AR,NaN,NaN,NaN
4,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810.0,NaN,US,US-AZ,Cordes,no,00AZ,NaN,00AZ,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43925,27242,ZYTL,large_airport,Zhoushuizi Airport,38.965698,121.539001,107.0,AS,CN,CN-21,Dalian,yes,ZYTL,DLC,NaN,NaN,http://en.wikipedia.org/wiki/Dalian_Zhoushuizi...,Dalian Air Base
43926,27243,ZYTX,large_airport,Taoxian Airport,41.639801,123.483002,198.0,AS,CN,CN-21,Shenyang,yes,ZYTX,SHE,NaN,NaN,http://en.wikipedia.org/wiki/Shenyang_Taoxian_...,NaN
43927,27244,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,42.882801,129.451004,624.0,AS,CN,CN-22,Yanji,yes,ZYYJ,YNJ,NaN,NaN,http://en.wikipedia.org/wiki/Yanji_Chaoyangchu...,NaN
43928,32753,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,NaN,AS,CN,CN-21,Shenyang,no,ZYYY,NaN,NaN,NaN,NaN,NaN


In [4]:
top

,City,FAA,IATA,ICAO,Airport,Role,Enplanements
0,Atlanta,ATL,ATL,KATL,Hartsfield-Jackson Atlanta International Airport,P-L,43130585
1,Chicago,ORD,ORD,KORD,Chicago O'Hare International Airport,P-L,32171831
2,Los Angeles,LAX,LAX,KLAX,Los Angeles International Airport,P-L,30528737
3,Dallas-Fort Worth,DFW,DFW,KDFW,Dallas/Fort Worth International Airport,P-L,27100656
4,Denver,DEN,DEN,KDEN,Denver International Airport,P-L,25241962
5,New York,JFK,JFK,KJFK,John F. Kennedy International Airport,P-L,22934047
6,San Francisco,SFO,SFO,KSFO,San Francisco International Airport,P-L,20038679
7,Houston,IAH,IAH,KIAH,George Bush Intercontinental Airport,P-L,19528631
8,Las Vegas,LAS,LAS,KLAS,McCarran International Airport,P-L,18996738
9,Phoenix,PHX,PHX,KPHX,Phoenix Sky Harbor International Airport,P-L,18907171
